In [ ]:
import sys
import importlib
from pprint import pprint

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import proplot as plot
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

sys.path.append('/Users/46h/Research/code/')
from accphys.tools import utils
from accphys.tools import beam_analysis as ba 
from accphys.tools import plotting as myplt
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled
from accphys.tools.accphys_utils import Sigma_from_twiss2D
from accphys.tools.accphys_utils import normalize
from accphys.tools.accphys_utils import possible_points
from accphys.tools.accphys_utils import normalized_Sigma
from accphys.tools.emittance_measurement import reconstruct
from accphys.tools.emittance_measurement import read_pta
from accphys.tools.emittance_measurement import plot_profiles
from accphys.tools.emittance_measurement import get_sig_xy

In [ ]:
plot.rc['grid.alpha'] = 0.04
plot.rc['figure.facecolor'] = 'white'

# RTBT emittance measurement data analysis 
> 05.31.2021

## Introduction 

### Method summary

Our goal is to reconstruct the transverse beam covariance matrix at position $s = s_0$:
<br>
<br>
$$
\Sigma_{0} = \begin{bmatrix}
    \langle{x^2}\rangle & \langle{xx'}\rangle & \langle{xy}\rangle & \langle{xy'}\rangle \\
    \langle{xx'}\rangle & \langle{{x'}^2}\rangle & \langle{yx'}\rangle & \langle{x'y'}\rangle \\
    \langle{xy}\rangle & \langle{yx'}\rangle & \langle{y^2}\rangle & \langle{yy'}\rangle \\
    \langle{xy'}\rangle & \langle{x'y'}\rangle & \langle{yy'}\rangle & \langle{{y'}^2}\rangle
\end{bmatrix}.
$$

We are taking $s_0$ to be the start of the RTBT. To do this, a set of $n$ wire-scanners can be placed at positions $\{s_i\} > s_0$ with $i = 1, ..., n$. A single measurement from the $i$th wire-scanner will produce the real-space moments of the beam at $s_i$: $\langle{x^2}\rangle_{i}$, $\langle{y^2}\rangle_{i}$, and $\langle{xy}\rangle_{i}$. Without space charge, the transfer matrix $M_{s_0 \rightarrow s_i} = M_i$ is known. The moments at $s_0$ are then related to those at $s_i$ by

$$\Sigma_i = M_i \Sigma_{0} {M_i}^T.$$ This gives three equations relating the measured moments at $s_i$ and the unknown moments at $s_0$: <br>

$$
\begin{align}
    \langle{x^2}\rangle_i &= 
        m_{11}^2\langle{x^2}\rangle_{0} 
      + m_{12}^2\langle{x'^2}\rangle_{0} 
      + 2m_{11}m_{22}\langle{xx'}\rangle_{0} ,\\
    \langle{y^2}\rangle_i &= 
        m_{33}^2\langle{y^2}\rangle_{0} 
      + m_{34}^2\langle{y'^2}\rangle_{0} 
      + 2m_{33}m_{34}\langle{yy'}\rangle_{0} ,\\
    \langle{xy}\rangle_i &= 
        m_{11}m_{33}\langle{xy}\rangle_{0} 
      + m_{12}m_{33}\langle{yx'}\rangle_{0} 
      + m_{11}m_{34}\langle{xy'}\rangle_{0} 
      + m_{12}m_{34}\langle{x'y'}\rangle_{0} ,
\end{align}
$$

where $m_{jk}$ are the elements of the transfer matrix. Taking 3 measurements with different optics settings between $s_0$ and $s_i$ (and therefore different transfer matrices) gives the 10 equations necessary to solve for $\Sigma_0$; however, real measurements will be noisy, so it is better to take more measurements if possible. Given $N$ measurements, we can form a $3N \times 1$ observation array $\mathbf{b}$ from the measured moments and a $3N \times 10$ coefficient array $\mathbf{A}$ from the transfer matrix such that

$$\begin{align} \mathbf{A \sigma}_0 = \mathbf{b},\end{align}$$ 

where $\mathbf{\sigma}_0$ is a $10 \times 1$ vector of the moments at $s_0$. There are 5 wire-scanners in the RTBT which operate simultaneously, so using all of them increases the size of the coefficient array to $15N \times 10$. We then choose $\mathbf{\sigma}_0$ such that $|\mathbf{A\sigma}_0 - \mathbf{b}|^2$ is minimized. 

### Details of this run

Regular production settings were used, i.e., the beam should be somewhat square in real space and have no cross-plane correlations. Thus, the four-dimensional emittance is expected to be the product of the two-dimensional apparent emittances. I don't remember the number of turns that were accumulated — maybe around 250. 

The phase advances were measured from the start of the RTBT to the last wire-scanner — WS24. Ideally, these phase advances are varied linearly in a 180 degree range centered on the default phase advances. We chose to cover this range in 12 steps; however, we didn't have time to do all 12 measurements. Therefore, instead of [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], we only did [0, 1, 3, 7, 11]. This should still be sufficient to reconstruct the covariance matrix. We also took a measurement at the default machine settings, but we can't use this in the reconstruction: I didn't record the field settings and can't get them now because the machine is down.

#### Note
We've changed our method since this study. The optics don't need to be changed so much; around 30 degrees at one wire-scanner will do. Furthermore, only the power supply on QH18 and QV19 need to be varied to set the phase advances. So, we should take these results with a grain of salt.

The profiles also look quite a bit different than the production documenation scans that we've analyzed. I think this is because our beam wasn't fully accumulated.

I also need to clean up this notebook move as much as possible to modules.

In [ ]:
scan_indices = [0, 1, 3, 7, 11]

## View parameters at each scan index 

In [ ]:
phases = np.loadtxt('phases.dat')
twiss_list = [np.loadtxt('./twiss_{}.dat'.format(i)) 
              for i in scan_indices + ['default']]

In [ ]:
fig, ax = plot.subplots(figsize=(4, 2))
ax.plot(np.degrees(phases[:, 0]), marker='.', alpha=0.1)
ax.plot(np.degrees(phases[:, 1]), marker='.', alpha=0.1)
ax.format(cycle='colorblind')
ax.scatter(scan_indices, np.degrees(phases[scan_indices, 0]), marker='.', s=50)
ax.scatter(scan_indices, np.degrees(phases[scan_indices, 1]), marker='.', s=50)
ax.legend(labels=[r'$\mu_x$', r'$\mu_y$']);
ax.format(xlabel='Scan index', ylabel='[degrees]', xticks=range(12), xtickminor=False,
          title='Phase advance at WS24 at each optics setting', grid=False)

In [ ]:
fig, ax = plot.subplots(figsize=(5, 2))
for twiss in twiss_list:
    pos, mu_x, mu_y, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.format(cycle='colorblind')
    ax.plot(pos, mu_x)
    ax.plot(pos, mu_y)
ax.legend(labels=[r'$\mu_x$', r'$\mu_y$'], ncols=1);
ax.format(ylabel=r'Phase adv. / 2$\pi$', xlabel='Position [m]',
          title='Phase advances at each optics setting')

In [ ]:
fig, ax = plot.subplots(figsize=(5, 2))
for twiss in twiss_list:
    pos, mu_x, mu_y, alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = twiss.T
    ax.format(cycle='colorblind')
    ax.plot(pos, beta_x)
    ax.plot(pos, beta_y)
ax.legend(labels=[r'$\beta_x$', r'$\beta_y$'], ncols=1);
ax.format(ylabel='[m]', xlabel='Position [m]', 
          title='Beta functions at each optics setting')

## Process wire-scanner data 

### Collect files 

In [ ]:
filenames_and_indices = []
for filename in utils.list_files('./', join=False):
    if not filename.startswith('Wire') or 'default' in filename:
        continue
    index = int(filename.split('.pta')[0].split('_')[-1])
    filenames_and_indices.append([filename, index])
filenames_and_indices = sorted(filenames_and_indices, key=lambda item: item[1])
filenames = [filename for (filename, index) in filenames_and_indices]
indices = [index for (filename, index) in filenames_and_indices]

We also took a measurement when the machine was at its default setting, but didn't save the transfer matrix. Add this measurement data for now.

In [ ]:
filenames.append('WireAnalysisFmt-2021.05.31_21.08.18_default.pta.txt')

In [ ]:
pprint(filenames)

### Extract profile data

Each profile measurement contains a series of positions $\{x_{i}\}$ and signal amplitudes $\{f(x_i)\}$ with $i$ = 1, ..., $N$. The Profile Tools and Analysis (PTA) application in OpenXAL actually analyzes the signals for us.

In [ ]:
measurements = [read_pta(filename) for filename in filenames]

In [ ]:
profile = measurements[0]['WS24']
display(profile.data)
display(profile.params)

### View profiles 

In [ ]:
ws_ids = ['WS20', 'WS21', 'WS23', 'WS24']
kws_raw = dict(legend=False, marker='.', ms=3, lw=0)
kws_fit = dict(legend=False, color='k', alpha=0.2, zorder=0)

#### Raw

In [ ]:
axes = plot_profiles(measurements, ws_ids, show_fit=False, kws_raw=kws_raw, kws_fit=kws_fit)
axes.format(leftlabels=scan_indices + ['default'])

#### Raw + fit

In [ ]:
axes = plot_profiles(measurements, ws_ids, show_fit=True, kws_raw=kws_raw, kws_fit=kws_fit)
axes.format(leftlabels=scan_indices + ['default'])

## Reconstruct beam moments

The covariance $\sigma_{xy}$ between *x* and *y* can be found using the beam size along the *u* axis normal to the diagonal wire.

<img src='diag_wire.png' width=200>

$$
\begin{align}
    u &= x\cos\phi + y\sin\phi \\
    \sigma_u^2 &= \langle{u^2}\rangle - \langle{u}\rangle^2 = \sigma_x^2\cos^2\phi + \sigma_y^2\sin^2\phi + 2\sigma_{xy}\sin\phi\cos\phi
\end{align}
$$

In [ ]:
diag_wire_angle = np.radians(-45.0)

Since we didn't store the default optics, we can't use that measurement in the reconstruction. 

In [ ]:
measurements = measurements[:-1]

Collect the measured beam moments at each wire-scanner. This is just for convenience.

In [ ]:
moments = {ws_id: [] for ws_id in ws_ids}
for ws_id in ws_ids:
    for measurement in measurements:
        profile = measurement[ws_id]
        sig_xx = profile.get_param('Sigma', dim='x')**2
        sig_yy = profile.get_param('Sigma', dim='y')**2
        sig_uu = profile.get_param('Sigma', dim='u')**2
        sig_xy = get_sig_xy(sig_xx, sig_yy, sig_uu, diag_wire_angle)
        moments[ws_id].append([sig_xx, sig_yy, sig_xy])
    moments[ws_id] = np.array(moments[ws_id])

In [ ]:
fig, axes = plot.subplots(ncols=4, figsize=(9, 2))
plt_kws = dict(marker='.')
for ws_id, ax in zip(ws_ids, axes):
    ax.plot(scan_indices, moments[ws_id][:, 0], **plt_kws)
    ax.plot(scan_indices, moments[ws_id][:, 1], **plt_kws)
    ax.plot(scan_indices, moments[ws_id][:, 2], **plt_kws)
axes.format(ylabel=r'[mm$^2$]', xlabel='Scan index', xtickminor=False, xticks=range(12), toplabels=ws_ids)
ax.legend(labels=[r'$\langle{x^2}\rangle$', r'$\langle{y^2}\rangle$', r'$\langle{xy}\rangle$'],
          ncols=1, loc=(1.02, 0));

Collect the transfer matrices at each wire-scanner as calculated by OpenXAL.

In [ ]:
transfer_mats = {ws_id: [] for ws_id in ws_ids}
for ws_index, ws_id in enumerate(ws_ids):
    if ws_id == 'WS02': # didn't take data from this wire-scanner
        continue
    for scan_index in scan_indices:
        filename = './transfer_mat_elems_{}.dat'.format(scan_index)
        matrix_elements = np.loadtxt(filename)[ws_index]
        transfer_mats[ws_id].append(matrix_elements.reshape((4, 4)))

Reconstruct the covariance matrix at the RTBT entrance.

In [ ]:
moments_list, transfer_mats_list = [], []
for ws_id in ws_ids:
    moments_list.extend(moments[ws_id])
    transfer_mats_list.extend(transfer_mats[ws_id])

In [ ]:
Sigma = reconstruct(transfer_mats_list, moments_list)
print('Sigma:')
print(Sigma)
alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = ba.get_twiss2D(Sigma)
twiss = (alpha_x, alpha_y, beta_x, beta_y)
eps_1, eps_2 = ba.intrinsic_emittances(Sigma)
print('eps_4D = {:.3f}'.format(np.sqrt(np.linalg.det(Sigma))))
print('eps_1, eps_2 = {:.3f}, {:.3f}'.format(eps_1, eps_2))
print('eps_x, eps_y = {:.3f}, {:.3f}'.format(eps_x, eps_y))
print('beta_x, beta_y = {:.3f}, {:.3f}'.format(beta_x, beta_y))
print('alpha_x, alpha_y = {:.3f}, {:.3f}'.format(alpha_x, alpha_y))

### Discussion 

The reconstructed beam has a larger emittance the $y$ direction than in the $x$ direction. The cross-plane moments are nonzero.

In [ ]:
correlation_matrix = utils.cov2corr(Sigma)
print(correlation_matrix)

I propose that we use a different definition of the coupling coefficient *C*. Prat and others define it as 

$$ C = \sqrt{\frac{\varepsilon_x\varepsilon_y}{\varepsilon_1\varepsilon_2}}.$$

$C = 1$ is no coupling, while $C \gt 1$ is nonzero coupling. In our case, ideally $\varepsilon_{4D} = \sqrt{\varepsilon_1\varepsilon_2} = 0$, so it would make more sense to define

$$ C = 1 - \sqrt{\frac{\varepsilon_1\varepsilon_2}{\varepsilon_x\varepsilon_y}},$$

so that $C$ is confined to the range [0, 1] (0 if no coupling, 1 if perfect rigid rotator).

In [ ]:
coupling_coeff = 1.0 - np.sqrt((eps_1 * eps_2) / (eps_x * eps_y))
print('Coupling coefficient = {}'.format(coupling_coeff))

Below are the rms ellipses defined by the covariance matrix: $x^T \Sigma x = 1$.

In [ ]:
myplt.rms_ellipses(Sigma, fill=True);

The $x$-$x'$ and $y$-$y'$ reconstructions don't depend on the diagonal wire. There's no way to check if the emittances are correct, but we can check the Twiss parameters. Below is a comparison of the rms ellipses.

In [ ]:
# Expected covariance matrix at RTBT entrance (from default optics)
alpha_x_exp = -1.22636996168 # [rad]
alpha_y_exp = 0.76481800616 # [rad]
beta_x_exp = 5.6562618021 # [m/rad]
beta_y_exp = 9.86908808294 # [m/rad]
twiss_exp = (alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp)
Sigma_exp = Sigma_from_twiss2D(alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp, eps_x, eps_y)

In [ ]:
def plot_lines(ax, transfer_mats, moments, dim='x', twiss=None, **plt_kws):  
    for transfer_mat, (sig_xx, sig_yy, sig_xy) in zip(transfer_mats, moments):
        h_pts, v_pts = possible_points(transfer_mat, sig_xx, sig_yy, dim, twiss)
        ax.plot(h_pts, v_pts, **plt_kws)

In [ ]:
def plot_reconstructed_phasespace(Sigma, Sigma_exp=None, twiss=None, scale=2.0):
    fig, axes = plot.subplots(ncols=2, sharex=False, sharey=False, figsize=(6, 2.5))

    # Plot reconstructed ellipses
    if twiss is not None:
        Sigma = normalized_Sigma(Sigma, *twiss)
    angle_xxp, cx, cxp = myplt.rms_ellipse_dims(Sigma, 'x', 'xp')
    angle_yyp, cy, cyp = myplt.rms_ellipse_dims(Sigma, 'y', 'yp')
    myplt.ellipse(axes[0], 2 * cx, 2 * cxp, angle_xxp, lw=2)
    myplt.ellipse(axes[1], 2 * cy, 2 * cyp, angle_yyp, lw=2)

    # Plot design ellipses
    plot_exp = Sigma_exp is not None
    if plot_exp:
        if twiss is not None:
            Sigma_exp = normalized_Sigma(Sigma_exp, *twiss)
        angle_xxp_exp, cx_exp, cxp_exp = myplt.rms_ellipse_dims(Sigma_exp, 'x', 'xp')
        angle_yyp_exp, cy_exp, cyp_exp = myplt.rms_ellipse_dims(Sigma_exp, 'y', 'yp')
        myplt.ellipse(axes[0], 2 * cx_exp, 2 * cxp_exp, angle_xxp_exp, lw=1, alpha=0.25)
        myplt.ellipse(axes[1], 2 * cy_exp, 2 * cyp_exp, angle_yyp_exp, lw=1, alpha=0.25)
        
    # Plot possible points at reconstruction location as lines. 
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    kws = dict(lw=1)
    for ws_id, color in zip(ws_ids, colors):
        plot_lines(axes[0], transfer_mats[ws_id], moments[ws_id], 'x', twiss, color=color, **kws)
        plot_lines(axes[1], transfer_mats[ws_id], moments[ws_id], 'y', twiss, color=color, **kws)

    # Legends
    custom_lines_1 = [Line2D([0], [0], color='black', lw=2)]
    labels_1 = ['calc']
    if plot_exp:
        custom_lines_1.append(Line2D([0], [0], color='black', lw=1, alpha=0.25))
        labels_1.append('exp')
    custom_lines_2 = [Line2D([0], [0], color=color) for color in colors[:4]]
    axes[1].legend(custom_lines_1, labels_1, ncols=1, loc=(1.02, 0.8), fontsize='small')
    axes[1].legend(custom_lines_2, ws_ids, ncols=1, loc=(1.02, 0), fontsize='small')

    # Formatting
    axes.format(grid=False, xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
    if twiss:
        axes.format(aspect=1)
        labels = [r"$x_n$ [mm]", r"$x'_n$ [mrad]", r"$y_n$ [mm]", r"$y'_n$ [mrad]"]
    else:
        labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]
        
    max_coords = 2.0 * np.sqrt(np.diag(Sigma))
    xmax, xpmax, ymax, ypmax = scale * max_coords
    axes[0].format(xlim=(-xmax, xmax), ylim=(-xpmax, xpmax), xlabel=labels[0], ylabel=labels[1])
    axes[1].format(xlim=(-ymax, ymax), ylim=(-ypmax, ypmax), xlabel=labels[2], ylabel=labels[3])
    return axes

In [ ]:
axes = plot_reconstructed_phasespace(Sigma, Sigma_exp)
axes.format(suptitle='Reconstructed phase space')

In [ ]:
axes = plot_reconstructed_phasespace(Sigma, twiss=twiss)
axes.format(suptitle='Reconstructed phase space (normalized by measured Twiss)')

It appears that changes to the optics did not change the phase advances in the correct way. I'm not sure exactly what happened.